In [1]:
%run stdPackages.ipynb
slides = True # print to slides format if True
out_folder = os.path.join(d['curr'], 'Misc', 'Figs')
d['data'] = os.path.join(d['curr'],'Misc','Data')
read = {'variables': ['Fundamentals', 'LoadVariables', 'TransmissionLines', 'GeneratorsVariables'],
        'maps': ['LoadMaps','GeneratorsMaps'],
        'variable2D': ['LoadVariation_E','LoadVariation_H','HourlyVariation'],
        'scalars': ['Scalars']}
db = dbFromWB(os.path.join(d['data'],'mGF_PH_ThellufsenEtAl2022.xlsx'), read)
readSets(db)

# Greenfield experiments with Danish-looking aggregate data

### 1. Descriptive

Let's start by plotting time series data. Collect all hourly variation in one dataframe:

In [2]:
load_E = db['LoadVariation_E'].rename_axis(index={'c_E':'hvt'})
load_E.index = load_E.index.set_levels('Load E_'+load_E.index.levels[-1], level='hvt')
load_H = db['LoadVariation_H'].rename_axis(index={'c_H':'hvt'})
load_H.index = load_H.index.set_levels('Load H_'+load_H.index.levels[-1], level='hvt')
df_H = pd.concat([db['CapVariation'].unstack('hvt'), load_E.unstack('hvt'), load_H.unstack('hvt')], axis=1).astype(float)

Split the samples into DK1 and DK2 time series':

In [3]:
DK1 = df_H.columns[df_H.columns.str.contains('DK1')].union(['SH'])
DK2 = df_H.columns[df_H.columns.str.contains('DK2')].union(['SH'])
df_DK1 = df_H[DK1]
df_DK2 = df_H[DK2]
df_DK1.columns = df_DK1.columns.str.strip('_DK1')
df_DK2.columns = df_DK1.columns.str.strip('_DK2')
print(df_DK1.corr().round(2).to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
hvt &    PV &  WNearShore &  WOffShore &  WOnShore &  Load E &  Load H &    SH \\
hvt        &       &             &            &           &         &         &       \\
\midrule
PV         &  1.00 &       -0.20 &      -0.21 &     -0.15 &    0.18 &   -0.27 &  0.49 \\
WNearShore & -0.20 &        1.00 &       0.77 &      0.91 &    0.28 &    0.39 & -0.22 \\
WOffShore  & -0.21 &        0.77 &       1.00 &      0.82 &    0.20 &    0.34 & -0.18 \\
WOnShore   & -0.15 &        0.91 &       0.82 &      1.00 &    0.26 &    0.40 & -0.18 \\
Load E     &  0.18 &        0.28 &       0.20 &      0.26 &    1.00 &    0.34 & -0.10 \\
Load H     & -0.27 &        0.39 &       0.34 &      0.40 &    0.34 &    1.00 & -0.35 \\
SH         &  0.49 &       -0.22 &      -0.18 &     -0.18 &   -0.10 &   -0.35 &  1.00 \\
\bottomrule
\end{tabular}



In [4]:
print(df_DK2.corr().round(2).to_latex())

\begin{tabular}{lrrrrrrr}
\toprule
hvt &    PV &  WNearShore &  WOffShore &  WOnShore &  Load E &  Load H &    SH \\
hvt        &       &             &            &           &         &         &       \\
\midrule
PV         &  1.00 &       -0.21 &      -0.13 &     -0.13 &    0.16 &   -0.23 &  0.48 \\
WNearShore & -0.21 &        1.00 &       0.87 &      0.82 &    0.20 &    0.29 & -0.21 \\
WOffShore  & -0.13 &        0.87 &       1.00 &      0.76 &    0.17 &    0.24 & -0.16 \\
WOnShore   & -0.13 &        0.82 &       0.76 &      1.00 &    0.28 &    0.36 & -0.20 \\
Load E     &  0.16 &        0.20 &       0.17 &      0.28 &    1.00 &    0.40 & -0.12 \\
Load H     & -0.23 &        0.29 &       0.24 &      0.36 &    0.40 &    1.00 & -0.34 \\
SH         &  0.48 &       -0.21 &      -0.16 &     -0.20 &   -0.12 &   -0.34 &  1.00 \\
\bottomrule
\end{tabular}



The marginal costs:

In [5]:
m = mGF_PH.mSimple(db)
m.preSolve()

What are the average costs per GJ supply if utilization was 100 \%? (the heat pumps are not included here; we can't assess them in the same way)
* If we install 1 GJ/h capacity, the plants produce ```m.hourlyCapFactors``` GJ throughout a year, with a marginal cost of ```mc```.
* The plant pays ```FOM``` and ```InvestCost_A``` for the 1 GJ/h capacity.
* For back-pressure plants, installing 1 GJ/h capacity can yield both production of heat and electricity. So, we scale costs with the amount of GJ produced. 

In [6]:
E2H_BP = mGF_PH.subsetIdsTech(db['E2H'], 'BP',db)
E2H_HP = mGF_PH.subsetIdsTech(db['E2H'], 'HP',db)
src = m.db['mc'].copy() # marginal costs of producing at capacity throughout a year
src.loc[E2H_BP.index] = src.loc[E2H_BP.index]/(1+1/E2H_BP) # scale SRC for back-pressure plants
invcosts = applyMult(m.db['InvestCost_A'], m.db['id2tech']).droplevel('tech') * 1000 / pdSum(m.hourlyCapFactors,'h')
invcosts.loc[E2H_BP.index] = invcosts.loc[E2H_BP.index]/(1+1/E2H_BP) # scale SRC for back-pressure plants
tc = invcosts.add(src,fill_value=0)

Map to technologies (cost structure of technologies is the same in DK1 and DK2)

In [7]:
src = applyMult(src, m.db['id2tech']).droplevel('id').groupby('tech').first()
invcosts = applyMult(invcosts, m.db['id2tech']).droplevel('id').groupby('tech').first()
tc = applyMult(tc, m.db['id2tech']).droplevel('id').groupby('tech').first()
df_Costs = pd.DataFrame({'Short run costs/GJ': src, 'Investment and fixed costs/GJ': invcosts, 'Unit cost': tc}).sort_values(by='Unit cost')

In [8]:
%%capture
one_graph()
fig, ax = plt.subplots(1,1,figsize = (14,8))
df_Costs.iloc[:,0:2].plot.bar(stacked=True,ax=ax)
ax.set_ylabel('$€/GJ$', labelpad=10);
ax.set_xlabel('Technology', labelpad=10);
fig.tight_layout();
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_Costs_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_Costs.pdf",edgecolor='k')

### 2. Create *design days approach*-like input data from full yearly samples

In [9]:
db_before = db.copy()

*Create representative days for each month instead:*

In [10]:
lm = [31,28,31,30,31,30,31,31,30,31,30,31] # number of days in the months
months = [i for l in [[j+1]*24*lm[j] for j in range(len(lm))] for i in l]
hours  = [j for l in [range(1,25)]*365 for j in l]
map_ = pd.MultiIndex.from_arrays([db['h'], hours, months], names = ['h','h_new','month'])
map2flat = pd.MultiIndex.from_arrays([pd.MultiIndex.from_arrays([months,hours]).to_flat_index(),months,hours], names = ['h','month','h_new']).unique()
map_flat = pd.MultiIndex.from_arrays([db['h'], pd.MultiIndex.from_arrays([months,hours]).to_flat_index()], names = ['h','h_new'])

*Use a version of aggregation that uses extreme values instead of averages (if a cluster group is below average --> use minimum observation, if a cluster group is above average --> use maximum observation):*

In [11]:
def aggregateV1(v, map_, map2flat):
    """ Map variable 'v' to hours and months. Flatten index to 1d"""
    v_gb = applyMult(v,map_).groupby(list(set(v.index.names).union(['h_new','month'])-set('h')))
    v_min,v_mean,v_max = v_gb.min()*v_gb.count(), v_gb.sum(), v_gb.max()*v_gb.count()
    mean_ = v_mean.groupby(list(set(v.index.names).union(['month'])-set('h'))).mean()
    v_new = pd.concat([v_min[(v_mean-mean_<=0).reorder_levels(v_mean.index.names)], v_max[(v_mean-mean_>0).reorder_levels(v_mean.index.names)]])
    v_new = applyMult(v_new, map2flat).droplevel(['h_new','month']).reorder_levels(v.index.names)
    return v_new * pdSum(v,'h') / pdSum(v_new,'h')
def aggregateV2(v, map_flat):
    """ Map variable 'v' to flat index of months/hours directly. """
    v_gb = applyMult(v,map_).groupby([k if k!='h' else 'h_new' for k in v.index.names])
    v_min,v_mean,v_max = v_gb.min()*v_gb.count(), v_gb.sum(), v_gb.max()*v_gb.count()
    mean_ = v_mean.groupby([k for k in v.index.names if k!='h']).mean()
    v_new = pd.concat([v_min[v_mean-mean_<=0], v_max[v_mean-mean_>0]]).rename_axis(index={'h_new':'h'})
    return v_new * pdSum(v,'h')/ pdSum(v_new,'h')

*Update symbols in database*

In [12]:
[db.__setitem__(k,aggregateV1(db[k], map_, map2flat)) for k in db.variableDomains('h')];
# [db.__setitem__(k,aggregateV2(db[k], map_flat)) for k in db.variableDomains('h')];
db['h'] = db['CapVariation'].index.levels[0] # update 'h' as well

### 3. Baseline model without aggregate capacity constraints

In [13]:
m = mGF_PH.mSimple(db)
m.solve()

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


**Generation capacity:**

In [14]:
cap = applyMult(pd.concat([applyMult(m.db['GeneratingCap_E'], db['id2tech']), applyMult(m.db['GeneratingCap_H'], db['id2tech'])]), db['id2g']).droplevel('id')
cap = cap.rename_axis(index={'g':None})
cap = cap.unstack().reindex(df_Costs.index)

In [15]:
%%capture
one_graph()
fig, ax = plt.subplots(1,1,figsize = (14,8))
cap.plot.bar(ax=ax);
ax.set_ylabel('TJ/h capacity', labelpad=10);
ax.set_xlabel('Technology', labelpad=10);
fig.tight_layout();
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_Baseline_GeneratingCap_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_Baseline_GeneratingCap.pdf",edgecolor='k')

*Plot **practical** capacity factors for active plants:*

In [16]:
capFactor_E = (pdSum(m.db['Generation_E'], 'h')/ pdNonZero(pdSum(m.hourlyGeneratingCap_E, 'h')) ).dropna() 
capFactor_H = (pdSum(m.db['Generation_H'], 'h')/ pdNonZero(pdSum(m.hourlyGeneratingCap_H, 'h')) ).dropna()
capFactor = applyMult(pd.concat([capFactor_E, capFactor_H]), db['id2tech']).droplevel('id')
capFactor = capFactor.rename_axis(index={'g':None})
capFactor = capFactor.unstack(0).reindex(df_Costs.index).dropna(how='all').fillna(0)

In [17]:
%%capture
one_graph()
fig, ax = plt.subplots(1,1,figsize = (14,8))
capFactor.plot.bar(ax=ax);
ax.set_ylabel('Practical capacity factor', labelpad=10);
ax.set_ylim([0,1]);
ax.set_xlabel('Technology', labelpad=10);
fig.tight_layout();
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_Baseline_PCF_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_Baseline_PCF.pdf",edgecolor='k')

*Plot residual demand curves:*

In [18]:
intermittent = mGF_PH.getTechs_i(['PV','WNearShore','WOffShore','WOnShore','SH'],m.db)
residualDemand_E = pdSum(m.hourlyLoad_E-pdSum(applyMult(rc_pd(m.hourlyGeneratingCap_E, intermittent), m.db['id2g']), 'id'), 'g')
residualDemand_H = pdSum(m.hourlyLoad_H-pdSum(applyMult(rc_pd(m.hourlyGeneratingCap_H, intermittent), m.db['id2g']), 'id'), 'g')

In [19]:
residualDemand_E = residualDemand_E.sort_values(ascending=False).reset_index(drop=True)
residualDemand_E.index = [i/(len(residualDemand_E)) for i in range(1, len(residualDemand_E)+1)]
residualDemand_E.at[0] = residualDemand_E.iloc[0]
residualDemand_H = residualDemand_H.sort_values(ascending=False).reset_index(drop=True)
residualDemand_H.index = [i/(len(residualDemand_H)) for i in range(1, len(residualDemand_H)+1)]
residualDemand_H.at[0] = residualDemand_H.iloc[0]

In [20]:
%%capture
mult_graphs()
nplots = 2
nrows = math.ceil(nplots/2)
fig, axes = plt.subplots(nrows, min(nplots, 2), figsize = (14, (6*nrows)));

# plot 1: RDC
ax = plt.subplot(nrows, min(nplots,2), 1)
seaborn.lineplot(data=residualDemand_E, ax = ax, linewidth=3);
ax.set_xlabel(r'Capacity Factor', labelpad = 5);
ax.set_ylabel(r'$TJ$', labelpad = 5);
ax.set_xlim([0, 1]);
ax.hlines(0,0,1,colors='k',linewidth=1,alpha=0.5)
ax.set_title('Residual Demand, E')

# plot 1: RDC
ax = plt.subplot(nrows, min(nplots,2), 2)
seaborn.lineplot(data=residualDemand_H, ax = ax, linewidth=3);
ax.set_xlabel(r'Capacity Factor', labelpad = 5);
ax.set_ylabel(r'$TJ$', labelpad = 5);
ax.set_xlim([0, 1]);
ax.hlines(0,0,1,colors='k',linewidth=1,alpha=0.5)
ax.set_title('Residual Demand, H')

fig.tight_layout()
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_Baseline_RDC_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_Baseline_RDC.pdf",edgecolor='k')

Plot the variation in system marginal costs (sort each of them separately):

In [21]:
msc_E = pd.DataFrame({k: m.db['marginalSystemCosts_E'].xs(k,level='g').sort_values(ascending=False).reset_index(drop=True) for k in m.db['marginalSystemCosts_E'].index.levels[0]})
msc_H = pd.DataFrame({k: m.db['marginalSystemCosts_H'].xs(k,level='g').sort_values(ascending=False).reset_index(drop=True) for k in m.db['marginalSystemCosts_H'].index.levels[0]})
msc_E.index = [i/(len(msc_E)) for i in range(1, len(msc_E)+1)]
msc_E.at[0] = msc_E.iloc[0]
msc_H.index = [i/(len(msc_H)) for i in range(1, len(msc_H)+1)]
msc_H.at[0] = msc_H.iloc[0]

In [22]:
%%capture
mult_graphs()
nplots = 2
nrows = math.ceil(nplots/2)
fig, axes = plt.subplots(nrows, min(nplots, 2), figsize = (14, (6*nrows)));

# plot 1: RDC
ax = plt.subplot(nrows, min(nplots,2), 1)
seaborn.lineplot(data=msc_E, ax = ax, linewidth=3);
ax.set_xlabel(r'Capacity Factor', labelpad = 5);
ax.set_ylabel(r'€/GJ', labelpad = 5);
ax.set_xlim([0, 1]);
ax.hlines(0,0,1,colors='k',linewidth=1,alpha=0.5)
ax.set_title('Marginal System Costs, E')

# plot 1: RDC
ax = plt.subplot(nrows, min(nplots,2), 2)
seaborn.lineplot(data=msc_H, ax = ax, linewidth=3);
ax.set_xlabel(r'Capacity Factor', labelpad = 5);
ax.set_ylabel(r'$€/GJ$', labelpad = 5);
ax.set_xlim([0, 1]);
ax.hlines(0,0,1,colors='k',linewidth=1,alpha=0.5)
ax.set_title('Marginal System Costs, H')

fig.tight_layout()
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_Baseline_MSC_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_Baseline_MSC.pdf",edgecolor='k')

In [23]:
db_baseline = m.db.copy()

### 4. Remove heat pumps

In [24]:
m.db['TechCap_H'] = (rc_pd(m.db['TechCap_H'], pd.Index(['HP'],name='tech')) * 0).combine_first(m.db['TechCap_H'])
m.solve()

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


*Compare capacities:*

In [25]:
cap_HP = applyMult(pd.concat([applyMult(m.db['GeneratingCap_E'], db['id2tech']), applyMult(m.db['GeneratingCap_H'], db['id2tech'])]), db['id2g']).droplevel('id')
cap_HP = cap_HP.rename_axis(index={'g':None})
cap_HP = cap_HP.unstack().reindex(df_Costs.index)
cap_compare = pd.DataFrame({'Baseline': cap.sum(axis=1), 'Without HP': cap_HP.sum(axis=1)})

In [26]:
%%capture
one_graph()
fig, ax = plt.subplots(1,1,figsize = (14,8))
cap_compare.plot.bar(ax=ax);
ax.set_ylabel('TJ/h capacity', labelpad=10);
ax.set_xlabel('Technology', labelpad=10);
fig.tight_layout();
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_HP_GeneratingCap_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_HP_GeneratingCap.pdf",edgecolor='k')

*Compare practical capacity factors for active plants:*

In [27]:
capFactor_E_HP = (pdSum(m.db['Generation_E'], 'h')/ pdNonZero(pdSum(m.hourlyGeneratingCap_E, 'h')) ).dropna() 
capFactor_H_HP = (pdSum(m.db['Generation_H'], 'h')/ pdNonZero(pdSum(m.hourlyGeneratingCap_H, 'h')) ).dropna()
capFactor_HP = applyMult(pd.concat([capFactor_E_HP, capFactor_H_HP]), db['id2tech']).droplevel('id')
capFactor_HP = capFactor_HP.rename_axis(index={'g':None})
capFactor_HP = capFactor_HP.unstack(0).reindex(df_Costs.index).dropna(how='all').fillna(0)
capFactor_compare = pd.DataFrame({'Baseline': capFactor.mean(axis=1), 'Without HP': capFactor_HP.mean(axis=1)})

In [28]:
%%capture
one_graph()
fig, ax = plt.subplots(1,1,figsize = (14,8))
capFactor_compare.plot.bar(ax=ax);
ax.set_ylabel('Practical capacity factor', labelpad=10);
ax.set_ylim([0,1]);
ax.set_xlabel('Technology', labelpad=10);
fig.tight_layout();
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_HP_PCF_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_HP_PCF.pdf",edgecolor='k')

*System costs, mean price, and emissions:*

In [29]:
costs_baseline = sum(m.hourlyLoad_H*m.db['MWP_LoadShedding_H'])+sum(m.hourlyLoad_E*m.db['MWP_LoadShedding_E'])-db_baseline['Welfare']
costs_hp = sum(m.hourlyLoad_H*m.db['MWP_LoadShedding_H'])+sum(m.hourlyLoad_E*m.db['MWP_LoadShedding_E'])-m.db['Welfare']
emissions_baseline = sum(db_baseline['Emissions'])
emissions_hp = sum(m.db['Emissions'])
mp_E_baseline = db_baseline['meanConsumerPrice_E'].mean()
mp_E_HP = m.db['meanConsumerPrice_E'].mean()
mp_H_baseline = db_baseline['meanConsumerPrice_H'].mean()
mp_H_HP = m.db['meanConsumerPrice_H'].mean()

In [30]:
df_compare = pd.DataFrame({'Baseline':  pd.Series([costs_baseline, emissions_baseline, mp_E_baseline, mp_H_baseline], index = pd.Index(['System costs','Emissions','mean price, E', 'mean price, H'], name = 'variable')),
                           'Without HP': pd.Series([costs_hp, emissions_hp, mp_E_HP, mp_H_HP], index = pd.Index(['System costs','Emissions','mean price, E', 'mean price, H'], name = 'variable'))})

Make the 'without HP' index 100:

In [31]:
df_compare = (df_compare.stack()/df_compare['Without HP']).unstack() * 100

In [32]:
%%capture
one_graph()
fig, ax = plt.subplots(1,1,figsize = (14,8))
df_compare.plot.bar(ax=ax, legend=False);
ax.set_ylabel("""Index 100 = 'Without HP' """, labelpad=10);
# ax.set_ylim([0,100]);
ax.set_xlabel(None);
fig.legend(df_compare.columns,loc=9,ncol=2,frameon=True)
fig.tight_layout();
fig.subplots_adjust(top=0.9);
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_HP_overview_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_HP_overview.pdf",edgecolor='k')

### 5. NIMBY constraints

Re-introduce heat pumps, but lower capacity for ```PV, SH, and WOnShore```:

In [33]:
m.db['TechCap_H'] = db_baseline['TechCap_H'].copy()
m.db['TechCap_E'] = pd.Series([2.5, 2.5, 8, 8], index = pd.MultiIndex.from_tuples([('DK1','PV'),('DK2','PV'), ('DK1','WOnShore'), ('DK2','WOnShore')], names = ['g','tech'])).combine_first(m.db['TechCap_E'])
m.db['TechCap_H'] = pd.Series([2,2], index = pd.MultiIndex.from_tuples([('DK1','SH'), ('DK2','SH')], names = ['g','tech'])).combine_first(m.db['TechCap_H'])
m.solve()

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


*Compare capacities:*

In [34]:
cap_NIMBY = applyMult(pd.concat([applyMult(m.db['GeneratingCap_E'], db['id2tech']), applyMult(m.db['GeneratingCap_H'], db['id2tech'])]), db['id2g']).droplevel('id')
cap_NIMBY = cap_NIMBY.rename_axis(index={'g':None})
cap_NIMBY = cap_NIMBY.unstack().reindex(df_Costs.index)
cap_compare = pd.DataFrame({'Baseline': cap.sum(axis=1), 'NIMBY': cap_NIMBY.sum(axis=1)})

In [35]:
%%capture
one_graph()
fig, ax = plt.subplots(1,1,figsize = (14,8))
cap_compare.plot.bar(ax=ax);
ax.set_ylabel('TJ/h capacity', labelpad=10);
ax.set_xlabel('Technology', labelpad=10);
fig.tight_layout();
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_NIMBY_GeneratingCap_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_NIMBY_GeneratingCap.pdf",edgecolor='k')

*Compare practical capacity factors for active plants:*

In [36]:
capFactor_E_NIMBY = (pdSum(m.db['Generation_E'], 'h')/ pdNonZero(pdSum(m.hourlyGeneratingCap_E, 'h')) ).dropna() 
capFactor_H_NIMBY = (pdSum(m.db['Generation_H'], 'h')/ pdNonZero(pdSum(m.hourlyGeneratingCap_H, 'h')) ).dropna()
capFactor_NIMBY = applyMult(pd.concat([capFactor_E_NIMBY, capFactor_H_NIMBY]), db['id2tech']).droplevel('id')
capFactor_NIMBY = capFactor_NIMBY.rename_axis(index={'g':None})
capFactor_NIMBY = capFactor_NIMBY.unstack(0).reindex(df_Costs.index).dropna(how='all').fillna(0)
capFactor_compare = pd.DataFrame({'Baseline': capFactor.mean(axis=1), 'NIMBY': capFactor_NIMBY.mean(axis=1)})

In [37]:
%%capture
one_graph()
fig, ax = plt.subplots(1,1,figsize = (14,8))
capFactor_compare.plot.bar(ax=ax);
ax.set_ylabel('Practical capacity factor', labelpad=10);
ax.set_ylim([0,1]);
ax.set_xlabel('Technology', labelpad=10);
fig.tight_layout();
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_NIMBY_PCF_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_NIMBY_PCF.pdf",edgecolor='k')

*System costs, mean price, and emissions:*

In [38]:
costs_NIMBY = sum(m.hourlyLoad_H*m.db['MWP_LoadShedding_H'])+sum(m.hourlyLoad_E*m.db['MWP_LoadShedding_E'])-m.db['Welfare']
emissions_NIMBY = sum(m.db['Emissions'])
mp_E_NIMBY = m.db['meanConsumerPrice_E'].mean()
mp_H_NIMBY = m.db['meanConsumerPrice_H'].mean()

In [39]:
df_compare = pd.DataFrame({'Baseline':  pd.Series([costs_baseline, emissions_baseline, mp_E_baseline, mp_H_baseline], index = pd.Index(['System costs','Emissions','mean price, E', 'mean price, H'], name = 'variable')),
                           'NIMBY'   :  pd.Series([costs_NIMBY, emissions_NIMBY, mp_E_NIMBY, mp_H_NIMBY], index = pd.Index(['System costs','Emissions','mean price, E', 'mean price, H'], name = 'variable'))})

Make the 'NIMBY' index 100:

In [40]:
df_compare = (df_compare.stack()/df_compare['NIMBY']).unstack() * 100

In [41]:
%%capture
one_graph()
fig, ax = plt.subplots(1,1,figsize = (14,8))
df_compare.plot.bar(ax=ax, legend=False);
ax.set_ylabel("""Index 100 = 'NIMBY' """, labelpad=10);
# ax.set_ylim([0,100]);
ax.set_xlabel(None);
fig.legend(df_compare.columns,loc=9,ncol=2,frameon=True)
fig.tight_layout();
fig.subplots_adjust(top=0.9);
if slides:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_NIMBY_overview_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mGF_PH_AAU_NIMBY_overview.pdf",edgecolor='k')